In [ ]:
import logging
import time
from datetime import datetime
from pathlib import Path
from shapely.geometry import Polygon, box
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import geohash
from functools import reduce

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [ ]:
cluster = LocalCluster(silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=2,
                       threads_per_worker=2,
                       memory_limit='5 GB')
client = Client(cluster)
client

In [3]:
base_path = Path('../../')
contiguous_us_bounding_box = box(-124.848974, 24.396308, -66.885444, 49.384358)

In [4]:
# load contiguous us data
df = dd.read_parquet(base_path / 'data/contiguous_us_geohash_sorted.parquet')
df.head(2)

,latitude,longitude
geohash,,
9hre04rq5gje,24.447,-124.443
9hreustkgy8k,24.591,-124.248


In [5]:
%%time
# let's take a 4 digit geohash (39.1km x 19.5km)
# increasing geohash precision inreases time to calculate polygon geohashes,
# but reduces the need for point in polygon post-filtering
geohash_precision = 4
df.index = df.index.str[:geohash_precision]
df = df.set_index(df.index, sorted=True)

CPU times: user 8.22 s, sys: 809 ms, total: 9.03 s
Wall time: 1min 4s


In [ ]:
df.to_parquet(base_path / 'data/contiguous_us_sorted_geohash4.parquet')